# 🦙 Fine-Tune LLaMA 3.2 3B with LoRA in Google Colab
This notebook fine-tunes `llama3.2:3b` using your `data.jsonl` file and saves the result for local Ollama use.

You'll:
1. Upload your `data.jsonl`
2. Install tools
3. Load and format data
4. Fine-tune using LoRA
5. Export to local machine

In [ ]:
# 🛠️ Step 1: Upload Your Verified Dataset
from google.colab import files
uploaded = files.upload()

In [ ]:
# ✅ Step 2: Install Required Packages
!pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
# ✅ Step 3: Preview JSONL Data
import json

# Extract filename dynamically from the uploaded dict
filename = next(iter(uploaded))  # gets the actual filename
print(f"Reading from uploaded file: {filename}")

with open(filename, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

print(f"✅ Loaded {len(data)} entries.")
print("🧪 First entry:\n", data[0])

In [ ]:
# ✅ Step 4: Convert to HuggingFace Dataset
from datasets import Dataset
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.05)

In [ ]:
# ✅ Step 5: Load LLaMA 3.2 3B Model
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, load_in_4bit=True, device_map="auto"
)

In [ ]:
# ✅ Step 6: Apply LoRA
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

In [ ]:
# ✅ Step 7: Format Data for Training
def format_example(example):
    return {
        "text": f"### Question:\n{example['prompt']}\n\n### Answer:\n{example['response']}"
    }
dataset = dataset.map(format_example)

In [ ]:
# ✅ Step 8: Tokenize
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
tokenized_dataset = dataset.map(tokenize)

In [ ]:
# ✅ Step 9: Train with Hugging Face Trainer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
args = TrainingArguments(
    output_dir="./llama3-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=1,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

In [ ]:
# ✅ Step 10: Export LoRA Model
!zip -r llama3-finetuned.zip ./llama3-finetuned
from google.colab import files
files.download("llama3-finetuned.zip")